<a href="https://colab.research.google.com/github/elizabethlconn/module-22-challenge/blob/main/Home_Sales_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install findspark
!pip install pyspark

In [2]:
import os
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# INSTALL SPARK AND JAVA
!apt-get update
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3-scala2.13.tgz
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!tar -xf spark-3.5.0-bin-hadoop3-scala2.13.tgz
!pip install -q findspark

# SETUP PATHS
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.5.0-bin-hadoop3-scala2.13'

# START A SPARKSESSION
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,630 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,566 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64

In [3]:
# IMPORT PACKAGES
from pyspark.sql import SparkSession
import time

# CREATE A SPARKSESSION
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# READ IN AWS S3 BUCKET TO DATAFRAME
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [5]:
# CREATE TEMPORARY VIEW OF DATAFRAME
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
df.createOrReplaceTempView('home_sales')
spark.sql("SELECT * FROM home_sales").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [7]:
# AVERAGE PRICE, FOUR BEDROOM HOUSE, SOLD PER YEAR, ROUNDED TWO DECIMAL PLACES
sql_query = """
SELECT YEAR(date), ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date); """

sql_query_df = spark.sql(sql_query)
sql_query_df.show()

+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [8]:
# AVERAGE PRICE OF A HOME FOR EACH YEAR THE HOME WAS BUILT, THREE BEDROOM, THREE BATHROOM, ROUNDED TWO DECIMAL PLACES
sql_query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built; """

sql_query_df = spark.sql(sql_query)
sql_query_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [9]:
# AVERAGE PRICE OF A HOME FOR EACH YEAR THE HOME WAS BUILT, THREE BEDROOM, THREE BATHROOM, TWO FLOORS, GREATER THAN OR EQUAL TO 2,000 SQFT, ROUNDED TWO DECIMAL PLACES
sql_query = """
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built; """

sql_query_df = spark.sql(sql_query)
sql_query_df.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# AVERAGE PRICE OF A HOME PER VIEW RATING, ROUNDED TWO DECIMAL PLACES
# HAVING AN AVERAGE HOME PRICE GREATER THAN OR EQUAL TO $350,000, ORDER BY DESCENDING VIEW RATING
# DETERMINE RUN TIME FOR QUERY
sql_query = """
SELECT view AS view_rating,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view_rating DESC;
"""

start_time = time.time()

# EXECUTE QUERY
sql_query_df = spark.sql(sql_query)

# SHOW RESULTS
sql_query_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
|         80| 991767.38|
+-----------+----------+
only showing top 20 rows

--- 0.9766943454742432 seconds ---


In [11]:
# CACHE TEMPORARY TABLE home_sales
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [12]:
# CHECK IF TABLE IS CACHED
spark.catalog.isCached('home_sales')

True

In [13]:
# USE CACHED DATA, RUN LAST QUERY ABOVE
# AVERAGE PRICE OF A HOME PER VIEW RATING, ROUNDED TWO DECIMAL PLACES
# HAVING AN AVERAGE HOME PRICE GREATER THAN OR EQUAL TO $350,000, ORDER BY DESCENDING VIEW RATING
# DETERMINE RUN TIME FOR QUERY AND COMPARE TO UNCACHED RUN TIME

start_time = time.time()

# EXECUTE QUERY USING CACHED DATA
sql_query = """
SELECT view AS view_rating,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view_rating DESC;
"""

sql_query_df = spark.sql(sql_query)

# DISPLAY RESULTS
sql_query_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
|         80| 991767.38|
+-----------+----------+
only showing top 20 rows

--- 0.6349678039550781 seconds ---


In [14]:
# PARTITION BY date_built FIELD ON FORMATTED PARQUET HOME SALES DATA
output_path = "/path/to/output/home_sales_parquet"
df.write.partitionBy("date_built").parquet(output_path)

In [15]:
# READ PARQUET FORMATTED DATA
parquet_df = spark.read.parquet(output_path)

In [16]:
# CREATE TEMPORARY TABLE FOR PARQUET DATA
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [17]:
# USING PARQUET DATAFRAME, RUN LAST QUERY ABOVE
# AVERAGE PRICE OF A HOME PER VIEW RATING, ROUNDED TWO DECIMAL PLACES
# HAVING AN AVERAGE HOME PRICE GREATER THAN OR EQUAL TO $350,000, ORDER BY DESCENDING VIEW RATING
# DETERMINE RUN TIME FOR QUERY AND COMPARE TO CACHED RUN TIME

start_time = time.time()

# EXECUTE QUERY USING PARQUET DATA
sql_query = """
SELECT view AS view_rating,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view_rating DESC;
"""
sql_query_df = spark.sql(sql_query)

# DISPLAY RESULTS
sql_query_df.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
|         80| 991767.38|
+-----------+----------+
only showing top 20 rows

--- 1.2460598945617676 seconds ---


In [19]:
# UNCACHED hom_sales TEMPORARY TABLE
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]